# **Bắt đầu code**

In [ ]:
### Import các thư viện cần thiết
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    RobertaForSequenceClassification,
    BertForSequenceClassification,
    TrainingArguments,
    Trainer,
    AutoConfig,
    EvalPrediction
)
from huggingface_hub import HfFolder, login
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

In [ ]:
### Đăng nhập vào Huggingface
# có thể không cần thiết phải đăng nhập (?) chỉ cần truyền token vào khi gọi hàm model.push_to_hub() là được
login(token = 'token của người dùng') # phải cung cấp 1 cái token của HF 

## **CONFIG**

In [ ]:
# FacebookAI/roberta-base; FacebookAI/roberta-large
# cardiffnlp/twitter-roberta-base; cardiffnlp/twitter-roberta-large-2022-154m
# Twitter/twhin-bert-base; Twitter/twhin-bert-large
''' đặt tên repo trên huggingface: 
prj-web-mining_{tên model}_{full-train hoặc sample-train}
'''

model_id = "cardiffnlp/twitter-roberta-large-2022-154m"  # tên huggingface của pretrained model, kéo về để đi finetune 
sampling_ratio = 1 # 0.0001 có sample tập train để thành tập train bé hơn không? # Note: để = 1 nếu không sample
num_epoch = 1

if sampling_ratio == 1:
    repository_id = f"shao2011/prj-web-mining_{model_id.split('/')[1]}_full-train" # tên huggingface để up checkpoint của model lên sau khi finetune xong
else:
    repository_id = f"shao2011/prj-web-mining_{model_id.split('/')[1]}_sample-train" # tên huggingface để up checkpoint của model lên sau khi finetune xong

# Các path của các file dataset
train_csv = '/kaggle/input/data-project-web-mining-2024-1/train/train.csv'
val_text_txt = '/kaggle/input/data-project-web-mining-2024-1/val/val_text.txt'
val_label_txt = '/kaggle/input/data-project-web-mining-2024-1/val/val_labels.txt'
test_text_txt = '/kaggle/input/data-project-web-mining-2024-1/test/test_text.txt'
test_label_txt = '/kaggle/input/data-project-web-mining-2024-1/test/test_labels.txt'

# file mapping (id, emoji, label)
mapping_txt = '/kaggle/input/data-project-web-mining-2024-1/mapping.txt'

print(model_id)
print(repository_id)

## **Load dataset (chuẩn bị dataset)**  
dataset phải là của class datasets.Dataset (chứ không phải Dataset của pytorch nhé)

In [ ]:
with open(mapping_txt, 'r') as f:
    lines = f.read().splitlines()
    lines = [line.split('\t') for line in lines]
    id2label = {int(line[0]): line[2] for line in lines}
    label2emoji = {line[2]: line[1] for line in lines}

# Hàm tính phân phối của các class trong 1 tập train/val/test -> để kiểm tra xem tập train có mất cân bằng dữ liệu không
def class_distributtion(dataset: Dataset, id2label: dict = id2label):
    count = dict()
    for i in range(len(dataset)):
        label = id2label[dataset[i]['label']] #note: dataset[i]['label'] là cái id, là số (còn label là string)
        if label not in count:
            count[label] = 0
        count[label] += 1

    total = len(dataset)
    for label in count:
        count[label] = round(count[label]/total*100,2)
        print(f"{label}: {count[label]} %")
        
    return count

In [ ]:
# Tạo dataset của tập train
df = pd.read_csv(train_csv)
train_dataset = Dataset.from_dict({
    'text': df['sentence'].to_list(),
    'label': df['label'].to_list()
})
print(train_dataset)
print(train_dataset[0])
print(train_dataset[1])
print(class_distributtion(train_dataset))

In [ ]:
# Tạo dataset của tập val
with open(val_text_txt, 'r') as f:
    val_text = f.read().splitlines()
with open(val_label_txt, 'r') as f:
    val_label = [int(i) for i in f.read().splitlines()]
    
val_dataset = Dataset.from_dict({
    'text': val_text,
    'label': val_label
})
print(val_dataset)
print(val_dataset[0])
print(val_dataset[1])
print(class_distributtion(val_dataset))

In [ ]:
# Tạo dataset của tập test
with open(test_text_txt, 'r') as f:
    test_text = f.read().splitlines()
with open(test_label_txt, 'r') as f:
    test_label = [int(i) for i in f.read().splitlines()]
    
test_dataset = Dataset.from_dict({
    'text': test_text,
    'label': test_label
})
print(test_dataset)
print(test_dataset[0])
print(test_dataset[1])
print(class_distributtion(test_dataset))

In [ ]:
# giải phóng bộ nhớ
df = None
val_text = None
val_label = None
test_text = None
test_label = None

In [ ]:
# def preprocess(text):
#     new_text = []
#     state1 = False
#     state2 = False
#     for t in text.split(" "):
#         if t.startswith('@') and len(t) > 1: 
#             state1 = True
#         if t.startswith('http'):
#             state2 = True
        
#         t = '@user' if t.startswith('@') and len(t) > 1 else t
#         t = 'http' if t.startswith('http') else t
#         new_text.append(t)
        
#     if state1 or state2:
#         print(state1, state2)
#         print('Câu gốc:', text)    
#         print('Câu mới:', " ".join(new_text))
              
#     return " ".join(new_text)

# for i in range(len(val_dataset)):
#     sen = test_dataset[i]['text']
#     new_sen = preprocess(sen)
    

### **Sample tập train nhỏ hơn**

In [ ]:
# sample tập train nhỏ hơn
train_dataset = train_dataset.shuffle().select(range(int(sampling_ratio * len(train_dataset))))
print(train_dataset)
print(class_distributtion(train_dataset))

## **Preprocessing**

### Tokenize các câu (sentence) bằng Tokenizer của model pretrained

In [ ]:
# Load lại tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id) # để sẵn: model = AutoModelForMaskedLM.from_pretrained("FacebookAI/roberta-base")

# Hàm tokenize: hàm này sử dụng cái Tokenizer của Roberata để tokenize cho các câu (biến mỗi một câu thành các token)
# Nó sẽ padding nếu câu ngắn hơn 256 và nó sẽ truncate (cắt xén) nếu câu dài hơn 256; 256 là cái max_length truyền vào.
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True, max_length=256)

In [ ]:
train_dataset = train_dataset.map(tokenize, batched=True, batch_size= len(train_dataset))
val_dataset = val_dataset.map(tokenize, batched=True, batch_size= len(val_dataset))
test_dataset = test_dataset.map(tokenize, batched=True, batch_size= len(test_dataset))

# print(train_dataset)
# print(val_dataset)
# print(test_dataset)

In [ ]:
print(train_dataset)
print(val_dataset)
print(test_dataset)

### Set dataset format

In [ ]:
# Set dataset format
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
val_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

train_dataset[0]

## **Xác định Cấu Hình (config) của model**
Mỗi model đều có 1 cấu hình (configuration - viết tắt là config), cấu hình xác định nhiều khía cạnh của model.  
Thử print cái cấu hình (config) sau đây thì sẽ thấy các khía cạnh nào.  
Cái cấu hình này chính là cấu hình của model được tạo ra (ở dòng `RobertaForSequenceClassification.from_pretrained(...)` )

In [ ]:
# Tạo object Config, nó biểu diễn cấu hình của model
config = AutoConfig.from_pretrained(model_id) # lấy cấu hình của model pretrained luôn (vào cái pretrained model trên HF: https://huggingface.co/FacebookAI/roberta-base/tree/main, rồi vào tab files and versions, rồi vào file config.json sẽ thấy)

# thêm cái id2label để phục vụ cho việc cấu hình các khía cạnh liên quan đến số class phân loại. 
# (nếu ko có, thì mặc định sẽ là binary classification)
config.update({'id2label': id2label}) # Phải đặt tên là id2label thì mới nhận diện được đây là các class output ra. 
print(config)

## **Tạo model và Load model**
- Cái cấu hình (config) được truyền vào để cấu hình lại model được tạo ra (ví dụ: `architecture` được dùng là gì, `hàm activate` là relu hay gelu, `id2label` như nào để còn tạo số nơ-ron của layer cuối trong bộ classifier, ...)
- Còn cái string "FacebookAI/roberta-base" chỉ là được dùng để xác định chỗ load trọng số pretrained thôi. (các trọng số pretrained sẽ được lấy giá trị từ cái checkpoint/repo tên là "FacebookAI/roberta-base").  


In [ ]:
if 'bert' in set(model_id.split('/')[1].split("-")):
    model = BertForSequenceClassification.from_pretrained(model_id, config = config)
else:
    model = RobertaForSequenceClassification.from_pretrained(model_id, config = config)
print('bert' in set(model_id.split('/')[1].split("-")))

print(model) # để ý: trong cái lớp (layer) out_proj xem cái out_features có = số class (= 20) không ?

## **Train model**

### Tạo Training Arguments và Trainer để train và cả evaluate model luôn
link doc của class Trainer: https://huggingface.co/docs/transformers/main_classes/trainer

In [ ]:
def macro_average_f1_score(evalPrediction: EvalPrediction):
    pred = np.argmax(evalPrediction.predictions, axis = 1)
    print(len(pred))
    print(pred)
    gold = list(evalPrediction.label_ids)
    
    report = classification_report(gold, pred, output_dict=True)
    f1_score = report['macro avg']['f1-score'] 
    acc_score = report['accuracy']
    return {'macro avg F1': f1_score, 'accuracy': acc_score}

In [ ]:
# TrainingArguments
training_args = TrainingArguments(
    output_dir=repository_id,
    
    num_train_epochs= num_epoch,
    learning_rate=5e-5,
    weight_decay=0.01,
    warmup_steps=500,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    
    evaluation_strategy="epoch",
    logging_dir=f"{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=10,
    save_strategy="epoch",

    load_best_model_at_end=True,
    metric_for_best_model = 'macro avg F1',
    save_total_limit=1,
    report_to="tensorboard",

    push_to_hub=True,
    hub_strategy="every_save",
    hub_model_id=repository_id,
    hub_token=HfFolder.get_token(),
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics = macro_average_f1_score
)

In [ ]:
trainer.train()

In [ ]:
test_result = trainer.evaluate(eval_dataset=test_dataset)
print("Kết quả eval trên test set")
for k in test_result:
    print(f"{k}: {test_result[k]}")